In [19]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder





ModuleNotFoundError: No module named 'sklearn'

In [15]:

input = [[2,3,3],[3,3,3]]
layer1 = Layer(3,2)
layer2 = Layer(2,1)
activation1 = layer1.forward(input)
activation2 = layer2.forward(activation1)
print(activation2)








[[6.66444422]
 [7.48024157]]


In [18]:
## Creds to 'https://www.geeksforgeeks.org/house-price-prediction-using-machine-learning-in-python/ for data preprocessing

 
s = (new_dataset.dtypes == 'object')
object_cols = list(s[s].index)
print("Categorical variables:")
print(object_cols)
print('No. of. categorical features: ',
      len(object_cols))

dataset = pd.read_csv('house.csv')
dataset.drop(['Id'],
             axis=1,
             inplace=True)

dataset['SalePrice'] = dataset['SalePrice'].fillna(
  dataset['SalePrice'].mean())

new_dataset = dataset.dropna()
OH_encoder = OneHotEncoder(sparse=False)
OH_cols = pd.DataFrame(OH_encoder.fit_transform(new_dataset[object_cols]))
OH_cols.index = new_dataset.index
OH_cols.columns = OH_encoder.get_feature_names()
df_final = new_dataset.drop(object_cols, axis=1)
df_final = pd.concat([df_final, OH_cols], axis=1)

MSSubClass      0
MSZoning        0
LotArea         0
LotConfig       0
BldgType        0
OverallCond     0
YearBuilt       0
YearRemodAdd    0
Exterior1st     0
BsmtFinSF2      0
TotalBsmtSF     0
SalePrice       0
dtype: int64

In [ ]:

## Class For each layer


class Layer:
    def __init__(self,inputneurons,neurons) -> None:
        
        self.weights = np.random.rand(inputneurons,neurons)
        self.biases = np.random.rand(neurons)
        self.runningsumw = np.zeros([inputneurons,neurons])
        self.runningsumb = np.zeros([inputneurons,neurons])
        self.activation = 
    def forward(self,input):
        return np.dot(input,self.weights)
    def backward(self,layerbefore):
        # Input is the partial derivative of the layer
        
        

    
class NeuralNetwork:
    def __init__(self,type,hiddenlayers= [10],batchsize = 32,epoch = 1) -> None:
        self.type = type
        self.batchsize = batchsize
        self.epoch = epoch
        self.hiddenlayers = hiddenlayers
        self.listoflayers = []
    def train(X,y):
        # Assert
        assert isinstance(X,pd.DataFrame)
        for column in df.columns:
            column_type = df[column].dtype
            
            assert column_type == 'float64' or column_type == 'int64'
        assert isinstance(y,pd.Series)
        sizeofinput = X.shape[1]
        for i in range(len(self.hiddenlayers) - 1):
            if i == 0:
                self.listoflayers.append(Layer(sizeofinput,self.hiddenlayer[i]))
            else:
                self.listoflayers.append(Layer(sizeofinput,self.hiddenlayerp[i - 1]))            
        if y.dtype == 'float64':
            self.listoflayers.append(Layer(self.hiddenlayer[-1],1))    
        for i in self.listoflayers:
            print('i.')


    












## Given some X and given some Y(Labeled )








